In [1]:
import glob
import random

from src.load_data import SingleProject, MultiProject

# Load Patterson Dataset

In [2]:
path = "../data/patterson"

single_projects = []

for file in glob.glob(path + "/*.rcp"):
    single_projects.append(SingleProject.read_instance(file))

In [3]:
single_projects = random.choices(single_projects, k=5)
instance = MultiProject.unify_projects(single_projects)

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [4]:
instance.n_jobs

146

In [5]:
instance.n_resources

3

In [6]:
len(instance.durations)

146

In [7]:
len(instance.successors)

146

In [8]:
len(instance.predecessors)

146

In [9]:
len(instance.required_resources)

146

# Run a simple heuristic

In [10]:
import numpy as np

In [11]:
max_time = np.sum(instance.durations)

In [12]:
active_jobs = set()
completed_jobs = set()
scheduled_jobs = set()

finishing_time = instance.n_jobs * [np.nan]
available_resources = max_time * [instance.resource_availability]
resource_transfers = np.zeros((instance.n_jobs, instance.n_jobs, instance.n_resources))
resource_stocks = np.zeros((instance.n_jobs, instance.n_resources))

t = 0

completed_jobs.add(0)
scheduled_jobs.add(0)
finishing_time[0] = 0

## No está en el algoritmo original (pongo todos los recursos en el primer job aunque no los necesita porque tienen que estar en algún sitio)
resource_stocks[0] = instance.resource_availability

# Set of elegible jobs at t=0
eligible_jobs = [
    i
    for i, pred in enumerate(instance.predecessors)
    if set(pred).issubset(completed_jobs)
]

# Iterate on increasing t until all jobs are assigned
while len(scheduled_jobs) < instance.n_jobs:
    if eligible_jobs:
        # Apply job rule
        # job with min latest finishing time (min LFT_MP)
        selected_job = eligible_jobs[np.argmin(instance.durations[eligible_jobs])]

        # Compute transfer feasible set for each resource type
        transfer_feasible_jobs = [
            (resource_stocks[job] > 0)
            * (finishing_time[job] + instance.transfer_times[job, selected_job] < t)
            for job in scheduled_jobs
        ]
        # Compute total supply os resources in t
        supply_of_resources = np.sum(
            transfer_feasible_jobs * resource_stocks[list(scheduled_jobs)], axis=0
        )

        # If supply is sufficient for all resources
        if (supply_of_resources > instance.required_resources[selected_job]).all():
            # Repeat until resource demand is satisfied
            while (
                resource_stocks[selected_job]
                < instance.required_resources[selected_job]
            ).any():

                # Apply resource transfer rule (min GAP)
                resource_needed = np.argmax(
                    instance.required_resources[selected_job]
                    - resource_stocks[selected_job]
                )
                feasible_jobs = [
                    s
                    for i, s in enumerate(scheduled_jobs)
                    if (transfer_feasible_jobs[i] * resource_stocks[s])[resource_needed]
                ]
                transfer_job = feasible_jobs[
                    np.argmin(
                        t
                        - (
                            np.array(finishing_time)[feasible_jobs]
                            + instance.transfer_times[feasible_jobs, selected_job]
                        )
                    )
                ]

                # Compute transferable amount of resource
                resource_transfers[transfer_job, selected_job, :] = np.min(
                    [
                        resource_stocks[transfer_job],
                        instance.required_resources[selected_job]
                        - resource_stocks[selected_job],
                    ],
                    axis=0,
                )
                # Update resource stocks
                resource_stocks[selected_job] += resource_transfers[
                    transfer_job, selected_job, :
                ]
                resource_stocks[transfer_job] -= resource_transfers[
                    transfer_job, selected_job, :
                ]

            # Schedule selected_job
            finishing_time[selected_job] = t + instance.durations[selected_job]
            scheduled_jobs.add(selected_job)
            active_jobs.add(selected_job)

            # Reduce available capacity of resources
            if instance.durations[selected_job] > 0:
                available_resources[
                    t : t + instance.durations[selected_job]
                ] -= instance.required_resources[selected_job]

        # Remove selected_job from elegible_jobs set
        eligible_jobs.remove(selected_job)

    # Increase current time
    t += 1

    # Compute set os jobs just completed
    jobs_just_completed = [job for job in active_jobs if finishing_time[job] < t]

    # Update set of active and completed jobs
    for job in jobs_just_completed:
        completed_jobs.add(job)
        active_jobs.remove(job)

    # Compute set of elegible tasks
    eligible_jobs = [
        job
        for job in set(range(instance.n_jobs)).difference(scheduled_jobs)
        if set(instance.predecessors[job]).issubset(completed_jobs)
        and (instance.required_resources[job] < available_resources[t]).all()
    ]

KeyboardInterrupt: 

In [ ]:
max(finishing_time)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
resources = np.concatenate([available_resources[: max(finishing_time)]])

fig, axes = plt.subplots(resources.shape[1], 1)

fig.suptitle("Resource usage")

for i in range(resources.shape[1]):
    axes[i].plot(resources[:, i])

plt.tight_layout()